# 鉅亨網爬蟲

In [ ]:
# 自動更新
'''
//3 minute default interval:
intervalInMilliSeconds =  180000

//include good ol' jQuery
var script = document.createElement('script');script.src = "https://code.jquery.com/jquery-3.4.1.min.js";document.getElementsByTagName('head')[0].appendChild(script);

//init and first click. Replace :nth(0) with whichever button you want to click. 0 is the first play button.
setTimeout(function(){$('colab-run-button:nth(0)').click()}, 1000)

//interval
setInterval(function(){ $('colab-run-button:nth(0)').click(); }, intervalInMilliSeconds)
'''

import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np
# # Google增強DataFrame的渲染和導出
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

# 連接到資料庫
conn = sqlite3.connect('anuenews.db')
# cursor object
cursor = conn.cursor()


uniqueSql = '''select * from news t'''


res = requests.get('https://news.cnyes.com/news/cat/tw_bull?exp=a')
soup = BeautifulSoup(res.text,"html.parser")


titletime = []
idBefore = []
id = []
content = []
  

# # 日期===========================================================================================================
times = soup.select('._67tN')
for time in times:
  head,mid,tail = time.find('time')['datetime'].partition('T') # partition‘T'之後分為頭部，中部，尾部三個
  titletime.append(head)
# print(len(titletime))

# # 內容============================
# titles = soup.select('._1xc2')
# for title in titles:
#   head,mid,tail = title.find('h3').text.partition(':')
#   id.append(head)
#   content.append(title.text)
# # print(len(id))
# # print(len(content))


# 內容===============================================================================================================
titles = soup.select('._1xc2')
# 1.去除特殊字符
for title in titles:
  head,mid,tail = title.find('h3').text.partition(':')
  if head[0] == '「':
    head2 = head.replace('「','') # replace（代替），將字符替換成
    head3 = head2.find('、') # 在第幾位

    if head3 < 0: # 沒有、號
      head2 = head2.split('」',1) # split（切割），切割為【」以前的內容】和【」以後的內容】
      head2 = head2[0]
      idBefore.append(head2)
    else: # 有、號
      head2 = head2[:head3]
      head2 = head2.split('」',1) 
      head2 = head2[0]
      idBefore.append(head2)
  else:
    idBefore.append(head)
  content.append(title.text)
# 2.去除【月營收】
for x in idBefore:
  x2 = x.find('月營收')
  if x2 < 0: # 沒有【月營收】
    id.append(x)
  else: # 有【月營收】
    x3 = x2 - 1
    x4 = x[:x3]
    id.append(x4)
# =================================================================================================================


data = {
  'date': titletime,
  'code': id,
  'content': content
}

df = pd.DataFrame(data)
# print(df)

# 今日新闻
newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期
df_mask = df['date']==newsToday            # 日期為今天的存檔
positions = np.flatnonzero(df_mask)
filtered_df=df.iloc[positions]


# for index, row in df.iterrows():
#     print(row['date'], row['code'],row['content'])

for index, row in filtered_df.iterrows():
  try:
    cursor.execute(
        """INSERT OR IGNORE INTO news 
          (date, code, content)
          values(?,?,?)""",
          (row['date'],
          row['code'],
          row['content']),
        )
    conn.commit()
    # 現在看來，commit()方法需要跟在增（insert）、刪（delete）、改（update）、查（select）的任何execute()語句後面。commit是把查詢語句提交到資料庫內，而不只是要向資料庫提交增、添的資料。
  except:
    pass

# pd.read_sql(uniqueSql,conn)


# 確認當日新聞 ========================================================================
dfToday = pd.read_sql(uniqueSql,conn)
df_maskToday = dfToday['date']==newsToday            # 日期為今天的存檔
# df_mask = df3['date']=='2022-08-29'           # 日期為今天的存檔
positionsToday = np.flatnonzero(df_maskToday)
filtered_dfToday=dfToday.iloc[positionsToday]
# ======================================================================================

filtered_dfToday

# 導出CSV

In [ ]:
import pandas as pd
import numpy as np
import sqlite3


conn2 = sqlite3.connect('/content/drive/MyDrive/anuenews/anuenews.db')
uniqueSql2 = '''select * from news t'''

df2 = pd.read_sql(uniqueSql2,conn2)
df3 = pd.DataFrame(df2, columns=['code','content','date'])

# 今日新闻
newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期
df_mask = df3['date']==newsToday            # 日期為今天的存檔
# df_mask = df3['date']=='2022-08-29'           # 日期為今天的存檔
positions = np.flatnonzero(df_mask)
filtered_df=df3.iloc[positions]

newsToday2 = pd.Timestamp.today().strftime("%Y%m%d")
filtered_df.to_csv(f"/content/drive/MyDrive/anuenews/csv/{newsToday2}鉅亨新聞.csv", encoding="big5hkscs",index=False)
# df3.to_csv(f"/content/drive/MyDrive/anuenews/csv/{newsToday2}鉅亨新聞.csv", encoding="big5hkscs",index=False)

print(f'{newsToday2}導出成功')

In [ ]:
# 创建资料库 ====================================================================
# import sqlite3


# # connecting to sqlite
# conn = sqlite3.connect('anuenews.db')
# # cursor object
# cursor = conn.cursor()

# # Creating table
# # 數據庫形態+content內容為不可重複
# news = """ CREATE TABLE news (
#     date DATE,
#     code TEXT,
#     content TEXT,
#     UNIQUE(content)
# ); """

# cursor.execute(news)
# ================================================================================

In [ ]:
# 删除资料 ====================================================================
# conn = sqlite3.connect('anuenews.db')
# cursor = conn.cursor()
# sql_delete_query = """DELETE from news where code = '〈美股早盤〉市場靜待Fed升息線索 主要指數開低'"""
# cursor.execute(sql_delete_query)
# conn.commit()
# pd.read_sql(uniqueSql,conn)
# ==============================================================================